<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/exemple_TensorDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exemple d'utilisation de torch.utils.data.TensorDataset : on a des bi,xi et on voudrait que f(bi)=xi

générer un dataset

In [1]:
import torch
import time
A = torch.rand(30,30)
B = torch.rand(10000,30)
X = torch.linalg.solve(A.expand(10000, -1, -1), B)

dataset = torch.utils.data.TensorDataset(B,X)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True,num_workers=3)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


apprendre un réseau

In [2]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet,self).__init__()
        self.f1 = torch.nn.Linear(30,64)
        self.f2 = torch.nn.Linear(64,128)
        self.f3 = torch.nn.Linear(128,30)

    def forward(self,x):
        x = torch.nn.functional.leaky_relu(self.f1(x))
        x = torch.nn.functional.leaky_relu(self.f2(x))
        return self.f3(x)

print("train loop")
t0 = time.time()
net = MyNet()
avgloss = []
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
for b,x in dataloader:
    xpred = net(b)
    loss = ((x-xpred)**2).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    avgloss.append(float(loss))
    if len(avgloss)==50:
        print("\t",sum(avgloss)/50)
        avgloss = []

print(time.time()-t0)

train loop


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


	 1.9573113799095154
	 0.9533476036787033
	 0.2262219974398613
	 0.13377402409911154
	 0.07987812243402004
	 0.05406187027692795
4.517619371414185


sans dataloader

In [3]:
print("train loop")
t0 = time.time()
net = MyNet()
avgloss = []
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
for i in range(10000//32):
    I = (torch.rand(32)*10000).long()
    b,x = B[I],X[I]
    xpred = net(b)
    loss = ((x-xpred)**2).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    avgloss.append(float(loss))
    if len(avgloss)==50:
        print("\t",sum(avgloss)/50)
        avgloss = []

print(time.time()-t0)

train loop
	 1.8767308855056763
	 1.0280935436487197
	 0.21374112904071807
	 0.12835330188274383
	 0.08580434218049049
	 0.05971808761358261
0.3272550106048584
